In [ ]:
import requests
import pandas as pd
import time
import math
from pathlib import Path


In [1]:
 pip install pytrends

In [2]:
from pytrends.request import TrendReq

# Inicializa pytrends
pytrends = TrendReq(hl='es-ES', tz=360)

# Define las palabras clave y el período de tiempo
kw_list = ['Taylor Swift']
timeframe = 'today 12-m'

# Realiza la búsqueda
pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='ES')

# Obtiene los datos de interés por región (comunidad autónoma)
interest_by_region = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)

# Calcula el porcentaje de popularidad de cada comunidad autónoma
total_interest = interest_by_region.sum()
popularity_percentage = (interest_by_region / total_interest) * 100

print(popularity_percentage)

                        Taylor Swift
geoName                             
Andalucía                   5.241605
Aragón                      4.586405
Canarias                    5.323505
Cantabria                   5.569206
Castilla y León             5.159705
Castilla-La Mancha          4.504505
Cataluña                    6.633907
Ceuta                       3.849304
Comunidad Valenciana        5.978706
Comunidad de Madrid         8.190008
Extremadura                 4.504505
Galicia                     6.306306
Islas Baleares              5.569206
La Rioja                    4.995905
Melilla                     3.276003
Navarra                     4.504505
País Vasco                  5.896806
Principado de Asturias      5.159705
Región de Murcia            4.750205


In [ ]:
pag = 0
url = 'https://app.ticketmaster.com/discovery/v2/events?apikey=4uZ7cKblFpFckdrfHMGrT2coHBKCiAjs&countryCode=ES&classificationName=music&locale=*&page=' + str(pag)
r = requests.get(url)

if r.status_code == 200:
  respuesta = r.json()
  total_pages = respuesta['page']['totalPages']
  print(total_pages)
  df_datos = pd.DataFrame(respuesta['_embedded']['events'])
  pag += 1
  while(pag < total_pages):
    url = 'https://app.ticketmaster.com/discovery/v2/events?apikey=4uZ7cKblFpFckdrfHMGrT2coHBKCiAjs&countryCode=ES&classificationName=music&locale=*&page=' + str(pag)
    r = requests.get(url)
    res = r.json()
    df_datos_mas = pd.DataFrame(res['_embedded']['events'])
    df_datos = pd.concat([df_datos, df_datos_mas], ignore_index=True)
    pag += 1
else:
  print('Error: ' + r.status_code)
  print(r.text)


len(df_datos)

39


766

In [ ]:
df_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             766 non-null    object
 1   type             766 non-null    object
 2   id               766 non-null    object
 3   test             766 non-null    bool  
 4   url              766 non-null    object
 5   locale           766 non-null    object
 6   images           766 non-null    object
 7   sales            766 non-null    object
 8   dates            766 non-null    object
 9   classifications  766 non-null    object
 10  promoter         765 non-null    object
 11  promoters        765 non-null    object
 12  priceRanges      763 non-null    object
 13  _links           766 non-null    object
 14  _embedded        766 non-null    object
 15  seatmap          83 non-null     object
dtypes: bool(1), object(15)
memory usage: 90.6+ KB


In [ ]:
print(df_datos['type'].unique())    # la columna type toma únicamente el valor 'event' -> eliminar
df_datos_menos_col = df_datos.drop(columns = ['test', 'url','images',
                                              'promoters', 'type', 'locale',
                                              '_links', 'seatmap'])
df_datos_menos_col

['event']


,name,id,sales,dates,classifications,promoter,priceRanges,_embedded
0,Jamie Cullum,Z698xZ2qZaa2L,{'public': {'startDateTime': '2023-11-30T16:56...,{'access': {'startDateTime': '2024-06-14T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6253', 'name': 'Tickex Experience, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Starlite', 'ty..."
1,Sheryl Crow,Z698xZ2qZaa1c,{'public': {'startDateTime': '2023-11-28T20:57...,{'access': {'startDateTime': '2024-06-21T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6253', 'name': 'Tickex Experience, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Starlite', 'ty..."
2,Aitana,Z698xZ2qZaa4g,{'public': {'startDateTime': '2024-01-09T12:29...,{'access': {'startDateTime': '2024-06-22T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6253', 'name': 'Tickex Experience, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Starlite', 'ty..."
3,Sheryl Crow - ALMA Festival Barcelona 2024,Z698xZ2qZaa8A,{'public': {'startDateTime': '2023-12-11T11:00...,{'access': {'startDateTime': '2024-06-24T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '3767', 'name': 'Concerts Studio, S.L.'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Poble Espanyol', 'type':..."
4,Glen Hansard - ALMA Festival Barcelona 2024,Z698xZ2qZaaFE,{'public': {'startDateTime': '2023-11-13T11:00...,{'access': {'startDateTime': '2024-06-25T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '3767', 'name': 'Concerts Studio, S.L.'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Poble Espanyol', 'type':..."
...,...,...,...,...,...,...,...,...
761,The Music of Hans Zimmer & others - A Celebrat...,Z698xZ2qZaFWl,{'public': {'startDateTime': '2023-08-09T15:26...,{'access': {'startDateTime': '2025-03-02T18:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty..."
762,"The Music of El Señor de los Anillos,El Hobbit...",Z698xZ2qZaFWs,{'public': {'startDateTime': '2023-08-09T15:28...,{'access': {'startDateTime': '2025-03-02T14:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty..."
763,The Magical Music of Harry Potter,Z698xZ2qZaaoh,{'public': {'startDateTime': '2023-12-07T16:17...,{'access': {'startDateTime': '2025-03-06T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Palau de Congressos de T...
764,Barcelona Bar Showroom,Z698xZ2qZaar2,{'public': {'startDateTime': '2024-02-13T16:26...,"{'start': {'localDate': '2024-03-17', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6341', 'name': 'BN GRUP 2019 S.L.'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Hyatt Regency Barcelona ...


In [ ]:
df_datos_menos_col.iloc[2]['classifications']

[{'primary': True,
  'segment': {'id': 'KZFzniwnSyZfZ7v7nn', 'name': 'Film'},
  'genre': {'id': 'KnvZfZ7vAkJ', 'name': 'Music'},
  'subGenre': {'id': 'KZazBEonSMnZfZ7vave', 'name': 'Music'},
  'family': False}]

In [ ]:
# Definir una función para extraer el valor startDateTime de cada fila
def obtener_start_datetime(fila):
    return fila['dates']['access']['startDateTime']

df_datos_menos_col.loc[0, 'startDateTime']= obtener_start_datetime(df_datos_menos_col.iloc[0])
df_datos_menos_col


,name,id,sales,dates,classifications,promoter,priceRanges,_embedded,startDateTime
0,Jamie Cullum,Z698xZ2qZaa2L,{'public': {'startDateTime': '2023-11-30T16:56...,{'access': {'startDateTime': '2024-06-14T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6253', 'name': 'Tickex Experience, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Starlite', 'ty...",2024-06-14T20:00:00Z
1,Sheryl Crow,Z698xZ2qZaa1c,{'public': {'startDateTime': '2023-11-28T20:57...,{'access': {'startDateTime': '2024-06-21T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6253', 'name': 'Tickex Experience, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Starlite', 'ty...",NaN
2,Aitana,Z698xZ2qZaa4g,{'public': {'startDateTime': '2024-01-09T12:29...,{'access': {'startDateTime': '2024-06-22T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6253', 'name': 'Tickex Experience, S.L.'}","[{'type': 'standard including fees', 'currency...","{'venues': [{'name': 'Auditorio Starlite', 'ty...",NaN
3,Sheryl Crow - ALMA Festival Barcelona 2024,Z698xZ2qZaa8A,{'public': {'startDateTime': '2023-12-11T11:00...,{'access': {'startDateTime': '2024-06-24T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '3767', 'name': 'Concerts Studio, S.L.'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Poble Espanyol', 'type':...",NaN
4,Glen Hansard - ALMA Festival Barcelona 2024,Z698xZ2qZaaFE,{'public': {'startDateTime': '2023-11-13T11:00...,{'access': {'startDateTime': '2024-06-25T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '3767', 'name': 'Concerts Studio, S.L.'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Poble Espanyol', 'type':...",NaN
...,...,...,...,...,...,...,...,...,...
761,The Music of Hans Zimmer & others - A Celebrat...,Z698xZ2qZaFWl,{'public': {'startDateTime': '2023-08-09T15:26...,{'access': {'startDateTime': '2025-03-02T18:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty...",NaN
762,"The Music of El Señor de los Anillos,El Hobbit...",Z698xZ2qZaFWs,{'public': {'startDateTime': '2023-08-09T15:28...,{'access': {'startDateTime': '2025-03-02T14:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty...",NaN
763,The Magical Music of Harry Potter,Z698xZ2qZaaoh,{'public': {'startDateTime': '2023-12-07T16:17...,{'access': {'startDateTime': '2025-03-06T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Palau de Congressos de T...,NaN
764,Barcelona Bar Showroom,Z698xZ2qZaar2,{'public': {'startDateTime': '2024-02-13T16:26...,"{'start': {'localDate': '2024-03-17', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6341', 'name': 'BN GRUP 2019 S.L.'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Hyatt Regency Barcelona ...,NaN


In [ ]:

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAME = "Taylor Swift"
DATE = "04-02-2024"  # Ajusta según sea necesario

# Construye la URL para solicitar datos de Last.fm
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={ARTIST_NAME}&api_key={LASTFM_API_KEY}&format=json"

# Realiza la solicitud a la API de Last.fm
response = requests.get(url)
data = response.json()

# Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
if "artist" in data and "stats" in data["artist"]:
    listeners = data["artist"]["stats"]["listeners"]
    print(f"Número de oyentes de {ARTIST_NAME} en {DATE}: {listeners}")
else:
    print(f"No se encontró información para {ARTIST_NAME} en la fecha {DATE}.")


Número de oyentes de Taylor Swift en 04-02-2024: 4495452


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Nombre del artista que deseas buscar
    artist_name = 'Taylor Swift'

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'
    search_params = {
        'q': artist_name,
        'type': 'artist',
    }

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista
    search_response = requests.get(search_url, params=search_params, headers=headers)
    search_data = search_response.json()

    # Obtener el ID del primer resultado (artista)
    if 'artists' in search_data and 'items' in search_data['artists'] and len(search_data['artists']['items']) > 0:
        artist_id = search_data['artists']['items'][0]['id']
        print(f'El ID del artista {artist_name} es: {artist_id}')

    else:
        print(f'No se encontró al artista {artist_name}')

else:
    print('Error al obtener el token de acceso')
    print(token_data)


El ID del artista Taylor Swift es: 06HL4z0CvFAxyc27GXpf02


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # ID del artista que deseas consultar (puedes obtenerlo de la API de búsqueda)
    artist_id = '06HL4z0CvFAxyc27GXpf02'

    # URL de la API de Spotify para obtener información del artista
    artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud para obtener información del artista
    artist_response = requests.get(artist_url, headers=headers)
    artist_data = artist_response.json()

    # Verificar si se obtuvo correctamente la información del artista
    if 'followers' in artist_data:
        followers_count = artist_data['followers']['total']
        print(f'El artista tiene {followers_count} oyentes mensuales.')

    else:
        print('Error al obtener información del artista')
        print(artist_data)

else:
    print('Error al obtener el token de acceso')
    print(token_data)


El artista tiene 101484352 oyentes mensuales.


In [ ]:
path = Path.cwd()   # 19/2/24 -> 39 paginas - 769 filas
print(path)
p = Path(path,"datos_19_02.csv")
df_datos.to_csv(p,index=False)
print(p)

/content
/content/datos_19_02.csv
